In [ ]:
%pip install tulipy
%pip install pynance

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import tulipy as ti
import pynance as pn

In [ ]:
# Directory containing your CSV files
data_dir = '../data/yfinance_data'  # Replace with the folder where your files are located
file_list = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
# Initialize an empty list to store DataFrames
all_data = []

# Process Each File
for file_name in file_list:
    file_path = os.path.join(data_dir, file_name)
    stock_data = pd.read_csv(file_path)

    # Ensure Date is a Datetime Object
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])

    # Add a column for the stock name (derived from file name)
    stock_name = os.path.splitext(file_name)[0]
    stock_data['Stock'] = stock_name

    # Append to the list
    all_data.append(stock_data)

# Concatenate all DataFrames into one
combined_data = pd.concat(all_data, ignore_index=True)

# Save Combined Data to CSV
output_file = '../data/combined_stock_data.csv'
combined_data.to_csv(output_file, index=False)
print(f"Combined data saved to {output_file}")

# Optional: Preview the combined data
print("Preview of Combined Data:")
print(combined_data.head())

In [ ]:
def visualize_indicators_in_grid(stock_data, stock_name):
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'{stock_name}: Technical Indicators', fontsize=16)

    # Close Price with SMA
    axes[0, 0].plot(stock_data['Close'], label='Close Price', alpha=0.75)
    axes[0, 0].plot(stock_data['SMA_20'], label='SMA 20', linestyle='--')
    axes[0, 0].set_title('Close Price with SMA')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Price')
    axes[0, 0].legend()

    # RSI
    axes[0, 1].plot(stock_data['RSI'], label='RSI', color='orange')
    axes[0, 1].axhline(70, color='red', linestyle='--', label='Overbought (70)')
    axes[0, 1].axhline(30, color='green', linestyle='--', label='Oversold (30)')
    axes[0, 1].set_title('RSI Indicator')
    axes[0, 1].set_xlabel('Date')
    axes[0, 1].set_ylabel('RSI Value')
    axes[0, 1].legend()

    # MACD
    axes[1, 0].plot(stock_data['MACD'], label='MACD', color='blue')
    axes[1, 0].plot(stock_data['MACD_signal'], label='Signal Line', color='red')
    axes[1, 0].set_title('MACD Indicator')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].set_ylabel('Value')
    axes[1, 0].legend()

    # Bollinger Bands
    axes[1, 1].plot(stock_data['Close'], label='Close Price', alpha=0.75)
    axes[1, 1].plot(stock_data['upper_band'], label='Upper Band', linestyle='--', color='red')
    axes[1, 1].plot(stock_data['middle_band'], label='Middle Band', linestyle='--', color='blue')
    axes[1, 1].plot(stock_data['lower_band'], label='Lower Band', linestyle='--', color='green')
    axes[1, 1].set_title('Bollinger Bands')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Price')
    axes[1, 1].legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


In [ ]:
def calculate_financial_metrics_with_pynance(stock_data, stock_name):
    print(f"Financial metrics for {stock_name}:")
    print(stock_data.describe())  # Summary statistics

    # Calculate Daily Returns
    stock_data['Daily_Returns'] = stock_data['Close'].pct_change()

    # Calculate Annualized Volatility (252 trading days assumed)
    annualized_volatility = stock_data['Daily_Returns'].std() * (252 ** 0.5)
    print(f"Annualized Volatility: {annualized_volatility:.2f}")

    # Visualize Daily Returns
    plt.figure(figsize=(10, 6))
    plt.plot(stock_data.index, stock_data['Daily_Returns'], label='Daily Returns', alpha=0.75)
    plt.title(f'{stock_name}: Daily Returns')
    plt.xlabel('Date')
    plt.ylabel('Returns')
    plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
    plt.legend()
    plt.show()

In [ ]:
combined_data['Date'] = pd.to_datetime(combined_data['Date'])
combined_data.set_index('Date', inplace=True)

for stock_name, stock_data in combined_data.groupby('Stock'):
    print(f"Processing stock: {stock_name}")

    # Drop rows with insufficient data for indicators
    if len(stock_data) < 26:  # Ensure enough data for MACD's long period
        print(f"Not enough data to calculate indicators for {stock_name}")
        continue

    # Ensure the data is sorted by date
    stock_data = stock_data.sort_index()

    # Convert Close prices to numpy array for tulipy
    close_prices = stock_data['Close'].values

    # Calculate Technical Indicators
    stock_data['SMA_20'] = pd.Series(ti.sma(close_prices, period=20), index=stock_data.index[-len(ti.sma(close_prices, period=20)):])
    stock_data['RSI'] = pd.Series(ti.rsi(close_prices, period=14), index=stock_data.index[-len(ti.rsi(close_prices, period=14)):])
    macd_data = ti.macd(close_prices, short_period=12, long_period=26, signal_period=9)
    stock_data['MACD'] = pd.Series(macd_data[0], index=stock_data.index[-len(macd_data[0]):])
    stock_data['MACD_signal'] = pd.Series(macd_data[1], index=stock_data.index[-len(macd_data[1]):])
    upper_band, middle_band, lower_band = ti.bbands(close_prices, period=20, stddev=2)
    stock_data['upper_band'] = pd.Series(upper_band, index=stock_data.index[-len(upper_band):])
    stock_data['middle_band'] = pd.Series(middle_band, index=stock_data.index[-len(middle_band):])
    stock_data['lower_band'] = pd.Series(lower_band, index=stock_data.index[-len(lower_band):])

    # Visualization in Grid
    visualize_indicators_in_grid(stock_data, stock_name)

    # Financial Metrics
    calculate_financial_metrics_with_pynance(stock_data, stock_name)

    # Save Processed Data
    output_file = f'../data/processed_{stock_name}.csv'
    stock_data.to_csv(output_file)
    print(f"Processed data saved for {stock_name} to {output_file}")
